In [1]:
from __future__ import division
import numpy as np
import pandas as pd

# Information Theory Problem Set 1

### 1

In [30]:
entropy = lambda x: -1*sum([y*np.log2(y) for y in x])

a

In [31]:
probs = [.1,.2,.5,.2]
entropy(probs)

1.7609640474436812

b

In [32]:
probs2 = [.1,.4,.5,.3]
entropy(probs2)

1.882053725693543

### 3b

In [33]:
#Optimal codeword length for a given probability
probs3 = [0.03125, 0.03125, 0.03125, 0.125, 0.25, 0.5, 0.015625, 0.015625]
for y in probs3:
    print y, np.log2(1/y)

0.03125 5.0
0.03125 5.0
0.03125 5.0
0.125 3.0
0.25 2.0
0.5 1.0
0.015625 6.0
0.015625 6.0


### 5a

In [2]:
import scipy.io as sio
data = sio.loadmat('spike_data.mat')['data']

In [5]:
data

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ..., 
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [3]:
import collections

In [102]:
data_ascodewords = [''.join(map(lambda x: str(x),y)) for y in data]

In [105]:
counts_codeword = collections.Counter(data_ascodewords)

In [108]:
counts_codeword_freq = {k:v/data.shape[0] for k,v in counts_codeword.iteritems()}

In [109]:
counts_codeword_freq

{'000': 0.7061,
 '001': 0.0381,
 '010': 0.0101,
 '011': 0.0407,
 '100': 0.0478,
 '101': 0.0029,
 '110': 0.03,
 '111': 0.1243}

In [111]:
entropy_spike = entropy(counts_codeword_freq.values())

In [112]:
entropy_spike

1.5488628651357339

# Information Theory Problem Set 2

2a

General formula:<br> 
I(X;Z|Y) = H(X,Y) + H(Y,Z) - H(X,Y,Z) - H(Y) where H is the entropy/joint entropy formula

In [113]:
data_df = pd.DataFrame(data,columns=['X','Y','Z'])

In [453]:
#Frequency of z and y
xy_counts = pd.crosstab(data_df['X'],data_df['Y'])
xy_freqs = xy_counts/data_df.shape[0]

In [454]:
yz_counts = pd.crosstab(data_df['Y'],data_df['Z'])
yz_freqs = yz_counts/data_df.shape[0]

In [455]:
xyz_counts = pd.crosstab(data_df['X'],[data_df['Y'],data_df['Z']])
xyz_counts_freqs = xyz_counts/data_df.shape[0]

In [436]:
def joint_entropy(cross_tab):
    """""
    calculates conditional entropy according to the formula:
    H(X,Y) = -1* sum for all x sum for all y [p(x,y)*log2(p(y|x))]
    """""
    return -1*(cross_tab*np.log2(cross_tab)).sum().sum()

In [458]:
I_X_Z_given_Y = joint_entropy(xy_freqs)+joint_entropy(yz_freqs)-joint_entropy(xyz_counts_freqs)-entropy(xy_freqs.sum())

In [459]:
I_X_Z_given_Y

2.7831628835905953e-05

The value of I(X;Z|Y) ~ 0. This means that knowing if we know information on Z|Y neuron, we gain no additional information about neuron X given Z|Y. Thus X and Z are conditionally independent given Y.

2b

I(X;Y|Z) = H(X,Z) + H(Y,Z) - H(X,Y,Z) - H(Z) where H is the entropy/joint entropy formula

In [468]:
xz_counts =  pd.crosstab(data_df['X'],data_df['Z'])
xz_freqs = xz_counts/data_df.shape[0]

In [472]:
I_X_Y_given_Z = joint_entropy(xz_freqs)+joint_entropy(yz_freqs)-joint_entropy(xyz_counts_freqs)-entropy(xz_freqs.sum())

In [473]:
I_X_Y_given_Z

0.1273202644492657

In [464]:
def calculate_information(dataframe,x,y):
    cross_tab_vals = pd.crosstab(dataframe[x],dataframe[y])/dataframe.shape[0]
    return entropy(cross_tab_vals.sum())+entropy(cross_tab_vals.sum(axis=1))-joint_entropy(cross_tab_vals)

In [465]:
i_x_z = calculate_information(data_df,'X','Z')
i_x_y = calculate_information(data_df,'X','Y')

In [466]:
i_x_y

0.29410398764218915

In [467]:
i_x_z

0.16681155482175902

I(X;Z|Y) < I(X;Y|Z). Calculating I(X;Z) and I(X;Y) shows that I(X;Z) < I(X;Y). This means that there is more shared information between X and Y than X and Z. Knowing information about X reduces the uncertainty of Y relative to the uncertainty of Z. 